In [1]:
from langchain_community.vectorstores import Pinecone
from langchain_openai import OpenAIEmbeddings
from langchain_community.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.chains import RetrievalQAWithSourcesChain
from langchain import prompts

# import SystemMessagePromptTemplate
from langchain.prompts import SystemMessagePromptTemplate

import pinecone
import os
import datetime
import json
import openai
import os
import pandas as pd
import pinecone
import re
from tqdm.auto import tqdm
from typing import List, Union
import zipfile

# Langchain imports
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import BaseChatPromptTemplate, ChatPromptTemplate
from langchain import SerpAPIWrapper, LLMChain
from langchain.schema import AgentAction, AgentFinish, HumanMessage, SystemMessage
# LLM wrapper
from langchain.chat_models import ChatOpenAI
from langchain import OpenAI
# Conversational memory
from langchain.memory import ConversationBufferWindowMemory

c:\Users\paulr\AppData\Local\pypoetry\Cache\virtualenvs\rag-demo-ggeqyuoK-py3.12\Lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
# get the api keys
open_ai_key = os.getenv('OPENAI_API_KEY') or 'OPENAI_API_KEY'

# init embeddings model
embed = OpenAIEmbeddings(
    model='text-embedding-ada-002',
)

# init vector store
PINECONE_API_KEY = os.getenv('PINECONE_RAGDEMO_API_KEY') or 'PINECONE_RAGDEMO_API_KEY'
PINECONE_ENVIRONMENT = os.getenv('PINECONE_RAGDEMO_ENV') or 'PINECONE_RAGDEMO_ENV'

pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENVIRONMENT
)

text_field = "text"
index_name = 'rag-demo'

# create index if it does not exist
if index_name not in pinecone.list_indexes():
    # we create a new index
    pinecone.create_index(
        name=index_name,
        metric='cosine',
        dimension=1536  # 1536 dim of text-embedding-ada-002
    )

# switch back to normal index for langchain
index = pinecone.Index(index_name)


vectorstore = Pinecone(
    index, embed, text_field
)

vector_db_index = pinecone.GRPCIndex(index_name)

# completion llm
llm_model = ChatOpenAI(
    # model_name='gpt-3.5-turbo',
    temperature=0.0
)

#qa_chain = RetrievalQAWithSourcesChain.from_chain_type(
qa_chain = RetrievalQA.from_chain_type(
    llm=llm_model,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

# conversational memory
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=3,
    return_messages=True
)

c:\Users\paulr\AppData\Local\pypoetry\Cache\virtualenvs\rag-demo-ggeqyuoK-py3.12\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [3]:
from langchain.agents import Tool

tools = [
    Tool(
        name='Knowledge Base',
        func=qa_chain.run,
        description=(
            'use this tool when answering astrophysics queries to get '
            'more information about the topic'
        )
    )
        Tool(
        name='[Knowledge Base]',
        func=qa_chain.run,
        description=(
            'use this tool when answering astrophysics queries to get '
            'more information about the topic'
        )
    )
]

In [4]:
from langchain.agents import initialize_agent

agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm_model,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=conversational_memory
)

c:\Users\paulr\AppData\Local\pypoetry\Cache\virtualenvs\rag-demo-ggeqyuoK-py3.12\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [5]:
agent("Tell me something about star formation")

c:\Users\paulr\AppData\Local\pypoetry\Cache\virtualenvs\rag-demo-ggeqyuoK-py3.12\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
{
    "action": "Knowledge Base",
    "action_input": "star formation"
}
Observation: The text provides some information about star formation, including the mass distribution of newly formed stars (initial mass function or IMF) and the influence of turbulent fragmentation in the process. However, it does not provide a comprehensive explanation of star formation.
Thought:{
    "action": "Final Answer",
    "action_input": "Star formation is the process by which dense regions within molecular clouds in interstellar space collapse under their own gravity and form stars. It involves the conversion of gas and dust into a protostar, which eventually becomes a main sequence star. The process of star formation is complex and involves various stages, including the collapse of a molecular cloud core, the formation of a protostar, and the eventual ignition of nuclear fusion in the core. The exact details of star formation are still an active area of researc

{'input': 'Tell me something about star formation',
 'chat_history': [],
 'output': 'Star formation is the process by which dense regions within molecular clouds in interstellar space collapse under their own gravity and form stars. It involves the conversion of gas and dust into a protostar, which eventually becomes a main sequence star. The process of star formation is complex and involves various stages, including the collapse of a molecular cloud core, the formation of a protostar, and the eventual ignition of nuclear fusion in the core. The exact details of star formation are still an active area of research in astrophysics.'}

In [6]:
agent("Make it one sentence")




> Entering new AgentExecutor chain...


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: Star formation is the process by which dense regions within molecular clouds collapse under gravity and form stars.